In [7]:
%matplotlib notebook

import pickle
import sys
import numpy as np
import import_ipynb
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.sparse.linalg import eigsh
from IPython.display import SVG, display
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.animation import FuncAnimation

# Adicionar o diretório raiz do projeto ao sys.path
project_root = Path().resolve().parent  
sys.path.append(str(project_root))
print("Project root added to sys.path:", project_root)

# Importando notebooks diretamente
try:
    import problem_statement as ps  
    from fem_processing import vectorial_matrices_assembly as assembly
    from fem_processing import master_domain as master
    from fem_pos_processing import graph_results as graph
    print("Modules imports were successful!")
except ModuleNotFoundError as e:
    print(f"Modules were not found: {e}")
except ImportError as e:
    print(f"Error in import: {e}")

Project root added to sys.path: C:\Users\adilt\OneDrive\05_GIT\PPGEE-2024.2-MEF
Modules imports were successful!


# Project 4: Rectangular Guide
# 3-noded Linear Triangular Elements, $P_1$

In [ ]:
FINITE_ELEMENT = ("Tetrahedron", 1)
BOUNDARY = [{'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}]
MATERIAL = [{'tag': 201, 'name': 'free_space', 'ur': 1, 'er': 1}]

# Create mesh from file geometry
mesh_data = ps.create_rectangular_cavity(FINITE_ELEMENT, BOUNDARY, MATERIAL, h=0.5, view_mesh=False)

# Applying physical properties to the mesh
mesh_data = ps.apply_physics(mesh_data)

# # Salvar mesh_data com pickle
# with open('mesh_data.pkl', 'wb') as f:
#     pickle.dump(mesh_data, f)

Malha salva em pre_processing/mesh/rectangular_cavity_domain_Tetrahedron1.msh
Model rectangular_cavity (3D)
Info     : 27 geometric entities
Info     : 2 Physical Groups
Info     : 45 nodes in total
Info     : 100 3-D elements in total


In [9]:
mesh_data['nodes']

{1: {'xg': (0.0, 0.0, 1.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 2: {'xg': (0.0, 0.0, 0.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 3: {'xg': (0.0, 1.0, 1.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 4: {'xg': (0.0, 1.0, 0.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 5: {'xg': (1.0, 0.0, 1.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 6: {'xg': (1.0, 0.0, 0.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 7: {'xg': (1.0, 1.0, 1.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 8: {'xg': (1.0, 1.0, 0.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 9: {'xg': (0.0, 0.0, 0.5),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'value': 0.0, 'name': 'nxE=0'}},
 10: {'xg': (0.0, 0.5, 1.0),
  'bc': {'tag': 101, 'type': 'Dirichlet', 'v

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2025.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.